In [201]:
import pandas as pd
import json
import datetime

# Hilfsfunktion für das Datum in den JSON Statements

In [202]:
# https://stackoverflow.com/questions/46906432/how-to-parse-datetime-that-ends-with-z
def string_to_datetime(date_string):
    date_format = '%Y-%m-%dT%H:%M:%S'
    date = datetime.datetime.strptime(date_string.split('.')[0], date_format)
    date = date + datetime.timedelta(microseconds = int(date_string.split('.')[1][:-1])/1000)
    return date

def datetime_to_string(date):
    date_format = '%Y-%m-%dT%H:%M:%S'
    date_string = date.strftime(date_format) + '.000Z'
    return date_string

# JSON Datei lesen und verändern

In [203]:
with open('moodle-dashboard.json') as file:
    data = json.load(file)
    
updated_data = data
    
# jede Beantwortung einer der datensouveränen Fragen heraussuchen
temp_data = []  
i = 0

while i < len(data):
    if 'Auswahl der Ã¼bermittelten Daten'  in str(data[i]['context']['contextActivities']):
        if 'answered' in str(data[i]['verb']['display']):
            temp_data.append(data[i])
    i += 1

In [204]:
# DataFrame mit allen wichtigen Informationen erstellen
df_data = []

i = 0
while i < len(temp_data):
    language = temp_data[i]['context']['language']
    df_data.append([temp_data[i]['actor']['name'],
                    temp_data[i]['actor']['mbox'],
                    temp_data[i]['object']['definition']['name'][language], 
                    temp_data[i]['result']['response'],
                    temp_data[i]['timestamp']
                   ])
    i += 1

df = pd.DataFrame(df_data, columns = ['Name', 'E-Mail', 'Frage', 'Antwort', 'Zeitpunkt'])

name_list = df['Name'].unique().tolist()
question_list = df['Frage'].unique().tolist()
df

,Name,E-Mail,Frage,Antwort,Zeitpunkt
0,Pandora Sykes,mailto:cursus.purus@icloud.ca,MÃ¶chtest du deinen vollen Namen Ã¼bermitteln?,Falsch,2023-09-14T08:35:52.000Z
1,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, in welcher Klasse d...",Falsch,2023-09-14T08:35:52.000Z
2,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, in welcheln FÃ¤cher...",Falsch,2023-09-14T08:35:52.000Z
3,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, welche Aufgabe du e...",Wahr,2023-09-14T08:35:52.000Z
4,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du die Zeit Ã¼bermitteln, zu der du ...",Wahr,2023-09-14T08:35:52.000Z
5,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, wann du die Aufgabe...",Falsch,2023-09-14T08:35:52.000Z
6,Pandora Sykes,mailto:cursus.purus@icloud.ca,MÃ¶chtest du deine Punktzahl Ã¼bermitteln?,Falsch,2023-09-14T08:35:52.000Z


In [213]:
for name in name_list:
    i = 0
    for question in question_list:
        times = df['Zeitpunkt'].unique().tolist()
        
        if len(times) == 1:
            count = 0
            if df.at[i, 'Antwort' == 'Falsch']:
                count += 1
                statement_ids = get_statement_ids(time[0], name)
                delete_statement_data(statement_ids, question, time[0], 0)
            i += 1
            #if count == len(question_list):
            #    delete_statement(get_statement_ids(time, name))
        else:
            t = 0
            while (t < len(times)-1):
                for time in times:
                    count = 0
                    if df.at[i, 'Antwort' == 'Falsch']:
                        count += 1
                        statement_ids = get_statement_ids(time[t], name)
                        delete_statement_data(statement_ids, question, time[t], 0)
                    t += 1
                    #if count == len(question_list):
                    #    delete_statement(get_statement_ids(time, name))
            for time in times:
                if df.at[i, 'Antwort' == 'Falsch']:
                    count += 1
                    statement_ids = get_statement_ids(time[t], name)
                    delete_statement_data(statement_ids, question, time[t], 0)
                i += 1

KeyError: False

In [210]:
# TODO: returns a string which gets deleted but does not conform to the path of where to find the deleting value
def get_value_to_be_deleted(question):
    if question == question_list[0]:
        return 'Name'
    elif question == question_list[1]:
        return 'Klasse'
    elif question == question_list[2]:
        return 'Fach'
    elif question == question_list[3]:
        return 'Aufgabe'
    elif question == question_list[4] or question_list[5]:
        return 'Zeitpunkt'      
    else:
        return 'Punktzahl'
        
def delete_statement(statement_ids):
    new_statements = []
    i = 0
    while i < len(updated_data):
        for statement_id in statement_ids:
            if updated_data[i]['id'] != statement_id:
                new_statements.append(updated_data[i])
        i += 1
    return new_statements

def delete_statement_data(statement_ids, question, start_time, end_time):
    i = 0
    for statement_id in statement_ids:
        for key, value in updated_data.items():
            if end_time == 0:
                if (statement_id == updated_data[i]['id'] & updated_data[i]['timestamp'] > start_time):
                    del value[get_value_to_be_deleted(question)]
                    updated_data[key] = value
            else:
                if (statement_id == updated_data[i]['id'] & (updated_data[i]['timestamp'] > start_time and updated_data[i]['timestamp'] < end_time)):
                    del value[get_value_to_be_deleted(question)]
                    updated_data[key] = value
            i += 1

def get_statement_ids(time, name):
    i = 0
    statement_ids = []
    while i < len(updated_data):
        if (updated_data[i]['timestamp'] > time) & (updated_data[i]['actor']['name'] == name):
            statement_ids.append(updated_data[i]['id'])
        i += 1
    return statement_ids


In [ ]:
# was muss potentiell gelöscht werden: 
# actor->name, 
# actor->mbox, 
# timestamp,
# result->completion, 
# result->success, 
# context->contextActivities->grouping[1] KLASSE
# context->contextActivities->grouping[2] QUIZ

# Statements in neue Datei schreiben

In [185]:
with open('new_statements.json', 'w') as new_file:
    json.dump(updated_data, new_file)

In [190]:
print('2023-09-14T08:35:52.000Z' > '2023-09-26T08:35:52.000Z')

False
